# ADD ELEMENTS TO THE ONTOLOGY OF EUCLID'S BOOK 1

In [ ]:
from __future__ import annotations

import pathlib
import sys

NOTEBOOK_DIR = pathlib.Path.cwd()
if '_NB_SYS_PATH_ADJUSTED' not in globals():
    sys.path.insert(0, str(NOTEBOOK_DIR))
    _NB_SYS_PATH_ADJUSTED = True

import pandas as pd
import rdflib

from modules import legacy_utils
from modules import rdf_utils

URIREF = rdflib.URIRef
RDF_TYPE = rdflib.RDF.type
RDFS_LABEL = rdflib.RDFS.label
SKOS_PREFLABEL = rdflib.SKOS.prefLabel
CONCEPT_CLASS = URIREF("https://www.foom.com/core#concept")
NAMESPACE = "<https://www.foom.com/core>"

In [ ]:
from modules import rdf_utils

def load_graph(input_ttl: pathlib.Path) -> rdflib.Graph:
    graph = rdf_utils.read_graph(str(input_ttl))
    return graph

#  read input ontology file
INPUT_TTL = NOTEBOOK_DIR.parent / "ontologies" / "ontology_euclid_book1.ttl"
g = load_graph(INPUT_TTL)


## CONCEPTUAL COMPONENTS OF POSTULATES AND COMMON NOTIONS

In [ ]:
# add contains_concepts relations to postulates and common notions
from modules import legacy_utils
from modules.import rdf_utils

INPUT_FILE = NOTEBOOK_DIR / "input" / "addenda_contains_concept.csv"
OUTPUT_FOLDER = NOTEBOOK_DIR / "ontologies"

def add_concept(
    g: rdflib.Graph,
    concept_string: str
):
    concept_label = concept_string.replace("_", " ")
    concept_iri = legacy_utils.create_iri(f"Concept: {concept_string}")
    g.add((URIREF(concept_iri), RDF_TYPE, CONCEPT_CLASS))
    g.add((URIREF(concept_iri), RDFS_LABEL, rdflib.Literal(f"Concept: {concept_label}")))
    g.add((URIREF(concept_iri), SKOS_PREFLABEL, rdflib.Literal(concept_label)))
    return g, concept_iri

def contains_concepts(
    g: rdflib.Graph,
    input_file: pathlib.Path = INPUT_FILE
):
    # read csv file
    df = pd.read_csv(input_file)
    concepts = set()
    for _, row in df.iterrows():
        subject_iri = URIREF(row['subject'])
        objects = {concept.strip() for concept in row['objects'].split(',')}
        for object_string in objects:
            object_iri = legacy_utils.create_iri(f"Concept: {object_string}")
            concepts.add(object_string)
            # add triples to the graph
            g.add((
                URIREF(subject_iri),
                URIREF("https://www.foom.com/core#contains_concept"),
                URIREF(object_iri)))
            g.add((
                URIREF(object_iri),
                URIREF("https://www.foom.com/core#is_concept_in"),
                URIREF(subject_iri)))
    return g, concepts


def write_csv(
        iris_df: pd.DataFrame, 
        *, 
        filename: str = "concepts.csv", 
        output_dir: pathlib.Path = OUTPUT_FOLDER
    ) -> pathlib.Path:
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / filename
    iris_df.to_csv(output_path, index=False)
    print(f"Saved iris to {output_path}")
    return output_path

def add_concepts_and_contains_concept(
    g: rdflib.Graph,
    input_file: pathlib.Path = INPUT_FILE
):
    g, concepts = contains_concepts(g, input_file)
    print(f"Added {len(concepts)} new concepts to the graph.")
    iris = set()
    for concept_string in concepts:
        g, concept_iri = add_concept(g, concept_string)
        iris.add(concept_iri)
    iris_df = pd.DataFrame({"iri": sorted(iris)})
    write_csv(iris_df)
    return g

# g = add_concepts_and_contains_concept(g, INPUT_FILE)

## CONCPETUAL COMPONENTS OF ALL CONCEPTS

In [ ]:
from modules import rdf_utils
INPUT_CSV = NOTEBOOK_DIR / "input" / "conceptual_components.csv"
OUTPUT_FOLDER = NOTEBOOK_DIR / "ontologies"

CONTAINS_CONCEPT = URIREF("https://www.foom.com/core#contains_concept")
IS_CONCEPT_IN = URIREF("https://www.foom.com/core#is_concept_in")

def add_conceptual_components(
    g: rdflib.Graph,
    input_csv: pathlib.Path = INPUT_CSV
):
    # read csv file with concepts and their conceptual components
    df = pd.read_csv(input_csv).fillna("")

    # iterate over rows and 
    # add triples <s, oai:contains_concept, o> and
    # <o, oai:is_concept_in, s>
    # to the graph
    for _, row in df.iterrows():
        subject_iri = URIREF(row['concept_iri'])
        if component_values := row['component']:
            components = {component.strip() for component in component_values.split(';')}
            for component_string in components:
                component_iri = URIREF(component_string)
                # add triples to the graph
                g.add((
                    subject_iri,
                    CONTAINS_CONCEPT,
                    component_iri))
                g.add((
                    component_iri,
                    IS_CONCEPT_IN,
                    subject_iri))
    
    print("Length of graph after adding conceptual components:", len(g))
    save_graph_with_timestamp(
        g, 
        output_dir=OUTPUT_FOLDER,)
    return g

g = add_conceptual_components(g, INPUT_CSV)

In [ ]:
from datetime import datetime
from pathlib import Path

def save_graph_with_timestamp(graph):
    """Serialize graph to a timestamped Turtle file in the study output folder."""
    output_dir = Path("/Users/matteobianchetti/Documents/Chen.Chen.1/Filosofia/Logica/Mathematics/Math_subjects/Computer_science/Git_repositories/GitHub/ontology-and-proofs/ontologies/elements_book1/study/output")
    output_dir.mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = output_dir / f"ontology_{timestamp}.ttl"
    graph.serialize(destination=output_path, format="turtle")
    return output_path
